In [11]:
# General libraries
import numpy as np
import pandas as pd
import random
import cv2
import os
import matplotlib.pyplot as plt

# Deep learning libraries
#import keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,Dropout,Activation,BatchNormalization
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

#Util Component 1: Confusion matrix report/Accuracy measures
from sklearn.metrics import accuracy_score, confusion_matrix

# disabling warnings
import logging
logging.getLogger('tensorflow').disabled = True #Jordan_note: Disable red warning lines seen at model architecture creation.

In [12]:
train_dir="/content/drive/MyDrive/machine learning projects/training set/Tomato leaf disease detection/tomato/train"
test_dir="/content/drive/MyDrive/machine learning projects/training set/Tomato leaf disease detection/tomato/val"

In [13]:
classes_train=os.listdir(train_dir)
classes_test=os.listdir(test_dir)
classes_train

['Tomato___Early_blight',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Bacterial_spot',
 'Tomato___healthy',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Target_Spot',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [14]:
train_idg=ImageDataGenerator(
    zoom_range=0.2, #the amount of zooming u need
    horizontal_flip=True, # Make a horizontal copy of image
    rescale=1.0/255.0, # Normalize the new images
    width_shift_range=0.10, # The percentage of Width shifitning
    height_shift_range=0.10, # The percentage of height shifitning
    shear_range=0.1 #Shear angle in counter-clockwise direction in degrees
)
train_flow=train_idg.flow_from_directory(
    train_dir,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    target_size=(256,256)[:10]
)

Found 10000 images belonging to 10 classes.


In [15]:
# Second for Testing data
test_idg=ImageDataGenerator(
    rescale=1.0/255.0
)
test_flow=test_idg.flow_from_directory(
    test_dir,
    class_mode='binary',
    color_mode='rgb',
    batch_size=32,
    target_size=(256,256)[:10]
)

Found 984 images belonging to 10 classes.


In [16]:
earlystop=EarlyStopping(patience=6)
learning_rate_reduction=ReduceLROnPlateau(
    monitor='val_acc',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001
)

In [17]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(3,3))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(3,3))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(3,3))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 84, 84, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 82, 82, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 27, 27, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 25, 25, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 128)       

In [19]:
callbacks=[earlystop,learning_rate_reduction]

In [20]:
model.fit(
    train_flow,
    epochs=10,
    validation_data=test_flow,
    callbacks=callbacks

)

Epoch 1/10
157/157 [==============================] - ETA: 0s - loss: 1.9274 - accuracy: 0.2910

ValueError: ignored

In [ ]:
losses=pd.DataFrame(model.history.history)

In [ ]:
losses.head()

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
losses[['accuracy','val_accuracy']].plot()
losses.plot()

In [ ]:
model_pred=model.predict(test_flow)